In [1]:
import numpy as np
import scipy.integrate as integrate
import sympy as sp

In [2]:
a, b, alpha, betta = 1.3, 2.2, 0, 5/6
f = lambda x: 4*np.cos(0.5*x)*np.exp(-5/4*x)+2*np.sin(4.5*x)*np.exp(x/8)+2
w = lambda x, i: (x-a)**(-alpha) * (b-x)**(-betta) * x**i
k, p = 15, 3


<img src="./Richardson/2.png">

In [3]:
def nutonkotes(a, b, points):
    """
    a - начало промежутка
    b - конец промежутка
    points - кол-во точек для разбиения промежутка [a, b]
    
    a < p1 < p2 < .. < pn < b
    """
    uzl = np.linspace(a, b, points) # Разбиваем промежуток на кол-во(points - кол-во отреков) равных отрезков. Uzl - массив с точками
    m = np.array([integrate.quad(w, a, b, args=(i)) for i in range(points)], dtype=np.float64) # считаем моменты
    M = m[:, 0];
    X = np.array([uzl**i for i in range(0, points)], dtype=np.float64); # составляем матрицу из узлов
    A = np.linalg.solve(X, M); # рещаем систему лин. уравнений, получаем коофиценты
    integral = np.sum([A[j]*f(uzl[j]) for j in range(points)]); # считаем инеграл, как сумму произведений соответсвующего коофицента
    # умноженного на значение функции в соответсвующем узле
    return integral

<img src="./Richardson/1.png">

In [5]:
def getIntegral(a, b, h):
    """
    Возвращает интеграл на промежутке [a,b] с шагом h.
    """
    h = int((b-a) / h)
    uzl = np.linspace(a, b, h)
    return np.array([nutonkotes(uzl[i], uzl[i+1], 3) for i in range(h-1)]).sum()

def Richardson(a, b, m, r, h1, L):
    """
    a - начало
    b - конец
    m - алг. точность
    r - размерность системы
    h1 - начальный шаг
    L - величина во сколько раз уменьшается шаг
    """
    h = np.array([h1 / (L**i) for i in range(r+1)]) # создаем вектор шагов длиной r+1. h1, h1/L, h1/L^2, h1/L^3 ..., h1/L^r
    S = - np.array([getIntegral(a, b, h[i]) for i in range(r+1)]) # считаем интегралы для каждого шага
    
    H = np.array([h ** (m+i) for i in range(r)]) # развертываем вектор h в матрицу размерности (r+1 x r).
    # 1ая строка матрицы равна h^m
    # 2ая строка матрицы h^(m+1)
    # ...
    # rая строка матрицы h^(m+r-1)
    ones = - np.ones((r+1)) # создаем вектор размерности r+1 из -1 
    H = np.append(H, ones).reshape(r+1, r+1).T # присоединяем вектор из -1 к нашей матрице и транспонируем ее. Размерность
    # полученной матрицы (r+1, r+1)
    
    C = np.linalg.solve(H, S) # решаем систему получаем вектор C. В последнем элементе вектора C записан искомый интеграл
    return abs(np.dot(C[:r], H[0][:r])), C[r] # выводим погрешность и инеграл

# L, h1 = 2, 100 # задаем L и начальный шаг
m = 3 # алгебраическая точность
r = 1 # задаем начальное r, влияющее на размерность системы. r=1 => размерность (2, 2)
error, J = Richardson(a, b, m, r, (b-a)/h1, L) # Расчитываем погрешность и интеграл.
while error > 1e-6: # пока погрешность не ниже 10^(-6) увеличиваем размерность системы
    error, J = Richardson(a, b, m, r, (b-a)/h1, L)
    r += 1 # увеличиваем размерность системы
    print(f'r = {r}.  Погрешность: {error}   Интеграл: {J}')